# Impact of Covid19 on the Suicidal Rate

________________________________________________________________________________________________________

In [1]:
# Mute the warnings
import shutup; shutup.please()

# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import numpy as np
from scipy.stats import linregress
import requests
from pprint import pprint
import urllib
import json
from pandas import json_normalize
import scipy.stats as stats
import scipy.stats as st
from scipy.stats import linregress


#Import API key for worldbank to get the global income level
from api_keys import api_key

ModuleNotFoundError: No module named 'api_keys'

In [ ]:
# creating the file path for data files
#suicide rate data file from data.oecd.org 
csv_path = "csv_file.csv"
#suicide rate data file from Kaggle.com used to get the different age group
master_path = "master.csv"
#Suicide rate in data for USA collected from cdc.com
USA_Rate = "Suicide_rate_USA.csv"
##suicide rate data file from Kaggle.com used to get the income level
Homicide_gdp = "suicide_homicide_gdp.csv"



In [ ]:
# Read the data file 
csv_file = pd.read_csv(csv_path)
master = pd.read_csv(master_path)
USA_data = pd.read_csv(USA_Rate)
hom_gdp = pd.read_csv(Homicide_gdp)


In [ ]:
# displaying the data from file csv_file
csv_file.head()

In [ ]:
#narrowing down the data to years 2016 to 2021 
narrow_data = csv_file.loc[(csv_file["TIME"]<2021) & (csv_file["TIME"]>=2016)]
#creating dataframe with the columns LOCATION,TIME,INDICATOR ,Value,SUBJECT
narrow_1 = pd.DataFrame(narrow_data, columns=["LOCATION", "TIME", "INDICATOR", "Value", "SUBJECT"])
#print the dataframe narrow_1
narrow_1.head()


In [ ]:
#gouping the datafrmae TIME
group_by_1 = narrow_1.groupby(["TIME"])["Value"].mean()
group_by_1

In [ ]:
group_by_df = pd.DataFrame(group_by_1)
group_by_df

In [ ]:
df2=group_by_df.reset_index()
df2

In [ ]:
# creating bar graph to visualize the suicide rate for chosen years
plot = group_by_1.plot(kind="bar", align = "center")
plt.xlabel("Year")
plt.ylabel("Suicide/100k of the Population")
plt.title("Global Suicide Rate by Year (2016 to 2020)")
plt.savefig("Output/Fig1.png")

plot

In [ ]:
# plot line suicide rate vs years
plt.plot(group_by_df, color = "b")
plt.xlabel("Year")
plt.ylabel("Suicide Rate/100k of the Population")
plt.title("Global Suicide Rate by Year (2016 to 2020)")
plt.savefig("Output/Fig2.png")

#extracting suicide rate for individual years for ANOVA test
group0 = narrow_1[narrow_1["TIME"] == 2016]["Value"]
group1 = narrow_1[narrow_1["TIME"] == 2017]["Value"]
group2 = narrow_1[narrow_1["TIME"] == 2018]["Value"]
group3 = narrow_1[narrow_1["TIME"] == 2019]["Value"]
group4 = narrow_1[narrow_1["TIME"] == 2020]["Value"]

#One way ANOVA test
stats.f_oneway(group0, group1, group2, group3, group4)




In [ ]:
#display the dat from master file for the age analysis
master.head()

In [ ]:
#narrow down the data for the years 2017 to 2022
narrow_data_2 = master.loc[(master["year"]<2022) & (master["year"]>=2016)]
#getting the data which has tha suicide rate >0
narrow_data_3 = narrow_data_2.loc[(narrow_data_2["suicides/100k pop"]>0)]
narrow_data_3
#creating a new dataframe with the columns country,year,age,suicides/100k pop
narrow_data_4 = pd.DataFrame(narrow_data_3, columns=["country", "year", "age", "suicides/100k pop"])

#narrow_data_2.dropna()
narrow_data_4.head()

In [ ]:
#grouping by age and getting average for each age group
age_data = narrow_data_4.groupby(["age"])["suicides/100k pop"].mean()


In [ ]:
#data frame for the agegroup
age_data_df = pd.DataFrame(age_data)
age_data_df

In [ ]:
# pie plot for age group
explode = (0, 0, 0.1, 0, 0, 0)
age_data_df.plot.pie(y='suicides/100k pop', explode=explode,autopct = "%1.1f%%", shadow = "True", startangle = 250)
plt.title("Global Suicide Rate by Age Group (2016 to 2020)")
plt.axis("equal")
plt.savefig("Output/Fig3.png")
plt.show()

In [ ]:
# Sucide rate based on gender
Men=[]
Women=[]
#dropping tha data from narrow_1(csv_file) which has total count for bot men and women
narrow_1_Drop_TOT = narrow_1.loc[narrow_1["SUBJECT"]!="TOT"]

#  getting average suicide rate using group by both gender(SUBJECT)and TIME(Year)
narrow_Gender = narrow_1_Drop_TOT.groupby(["TIME","SUBJECT"])["Value"].mean()

#narrowing down to MEN 
Men_DF = narrow_1_Drop_TOT.loc[narrow_1_Drop_TOT["SUBJECT"]=="MEN"]
#narrowing down to WOMEN
women_DF =narrow_1_Drop_TOT.loc[narrow_1_Drop_TOT["SUBJECT"]=="WOMEN"]

#gettin Rate for Men and women in seperate series
Men = Men_DF.groupby(["TIME"])["Value"].mean()
Women = women_DF.groupby(["TIME"])["Value"].mean()
time = Men.index

#creating DataFrame
Gender = pd.DataFrame({'Men': Men,'Women': Women}, index=time)
#Plotting Bar graph
Gender.plot.bar(rot=0)

plt.title("Global Suicide Rate by Gender")
plt.xlabel("Gender (Men/Women) /Year")
plt.ylabel("Suicide Rate")
plt.legend( loc ="lower right")
plt.savefig("Output/Fig4.png")




In [ ]:
#Line plot for Both Men and Women Series

time = np.arange(2016, 2021, 1)
Men1, = plt.plot(time, Men, color="red", label="Men")
Women1, = plt.plot(time, Women, color="blue", label="Women")


plt.title("Global Suicide Rate by Gender (2016 to 2020)")
plt.xlabel("Year")
plt.ylabel("Suicide Rate/100k Population")
plt.legend(handles=[Men1, Women1], loc="best")
plt.grid()
plt.savefig("Output/Fig5.png")


In [ ]:
# Getting income level for different nations using API  api.worldbank.org

countries = []
country_income=[]
country_name =[]
income_level = []


#query

query_url = "http://api.worldbank.org/v2/countries?format=json"

country_data = requests.get(query_url).json()
countries = country_data[1]
#travesing the JSON to get the name and income level of all the country
for  country in countries:
        
    try:
        # Parse the JSON and retrieve data
        country_name.append(country['name'])
       
        income_level.append(country['incomeLevel']['value'])
        
       
    # If an error is experienced, skip the country
    except:
        print("country not found. Skipping...")
        pass
    
#creating a data frame with country and its income level              
country_income_df = pd.DataFrame({"Country": country_name, 
                                  "Income": income_level})
print(country_income_df)
print(f"Number of countries in the list: {len(countries)}")


In [ ]:
#nqlysing income level using  narrow_data_4(master.csv)
narrow_data_4

In [ ]:
hom_gdp.head()

hom_gdp_merged = pd.merge(hom_gdp, narrow_data_4, on="country")
hom_gdp_merged.head()
hom_gdp_merged_2019 = hom_gdp_merged.loc[(hom_gdp_merged["year_y"]==2019)]
hom_gdp_merged_2019
hom_gdp_merged_2019_1 = hom_gdp_merged_2019.groupby(["incomeLevel"])["Suicide mortality rate (per 100,000 population)"].mean()
hom_gdp_merged_2019_1


In [ ]:
hom_gdp_merged_2016 = hom_gdp_merged.loc[(hom_gdp_merged["year_y"]==2016)]
hom_gdp_merged_2016
hom_gdp_merged_2016_1 = hom_gdp_merged_2016.groupby(["incomeLevel"])["Suicide mortality rate (per 100,000 population)"].mean()
hom_gdp_merged_2016_1

In [ ]:
hom_gdp_merged_2017 = hom_gdp_merged.loc[(hom_gdp_merged["year_y"]==2017)]
hom_gdp_merged_2017
hom_gdp_merged_2017_1 = hom_gdp_merged_2017.groupby(["incomeLevel"])["Suicide mortality rate (per 100,000 population)"].mean()
hom_gdp_merged_2017_1

In [ ]:
hom_gdp_merged_2018 = hom_gdp_merged.loc[(hom_gdp_merged["year_y"]==2018)]
hom_gdp_merged_2018
hom_gdp_merged_2018_1 = hom_gdp_merged_2018.groupby(["incomeLevel"])["Suicide mortality rate (per 100,000 population)"].mean()
hom_gdp_merged_2018_1

In [ ]:
hom_gdp_merged_2020 = hom_gdp_merged.loc[(hom_gdp_merged["year_y"]==2020)]
hom_gdp_merged_2020
hom_gdp_merged_2020_1 = hom_gdp_merged_2020.groupby(["incomeLevel"])["Suicide mortality rate (per 100,000 population)"].mean()
hom_gdp_merged_2020_1


In [ ]:
summary_df_1 = pd.DataFrame({" 2017": hom_gdp_merged_2017_1, " 2018": hom_gdp_merged_2018_1, "2019": hom_gdp_merged_2019_1, "2020": hom_gdp_merged_2020_1})

summary_df_1

In [ ]:
#plotting bar graph for Suicide rate vs year and income level - high,lower middle and upper middle
summary_df_2 = summary_df_1.transpose()
summary_df_2

summary_df_2.plot(kind = "bar")
plt.title("Global Suicide Rate by Income Level (2017 to 2020)")
plt.xlabel("Income Level")
plt.ylabel("Suicide Rate/100k of the Population")
plt.savefig("Output/Fig6.png")
summary_df_2

In [ ]:
#creating dataframe from the csv file USA_data

USA_by_year_df = pd.DataFrame(USA_data, columns=["YEAR","STATE","RATE","DEATHS"])
#filtering the year to 2016 -2021
USA_by_year_df= USA_by_year_df.loc[(USA_by_year_df["YEAR"]>=2016) & (USA_by_year_df["YEAR"]<=2021)]
#finding average suicide rate
state = USA_by_year_df.groupby(["YEAR"])["RATE"].mean()

#plotting bar graph
state.plot.bar(y= "RATE")

plt.title("United States Suicide Rate (2016 to 2021)")
plt.xlabel("Year")
plt.ylabel("Suicide Rate/100k Population")
plt.savefig("Output/Fig7.png")

USA_by_year_df

In [ ]:
#computing Linear regression for 4 sample states Texas,Florida,Colorodo and california

#Texas
USA_by_year_TX = USA_by_year_df.loc[USA_by_year_df["STATE"]=="TX"]
x=USA_by_year_TX["YEAR"]
y=USA_by_year_TX["RATE"]
correlation = st.pearsonr(x,y)
(slope, intercept, rvalue, pvalue, stderr) = linregress(x, y)
regress_values = x * slope + intercept
line_equation = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x, y,c="#1f77b4")#blue
plt.plot(x, y, "r-")


#Florida
USA_by_year_NY = USA_by_year_df.loc[USA_by_year_df["STATE"]=="FL"]
x=USA_by_year_NY["YEAR"]
y=USA_by_year_NY["RATE"]
correlation = st.pearsonr(x,y)
(slope, intercept, rvalue, pvalue, stderr) = linregress(x, y)
regress_values = x * slope + intercept
line_equation = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x, y,c="#ff7f0e")#orange
plt.plot(x, y, "r-")

#colorodo
USA_by_year_CO = USA_by_year_df.loc[USA_by_year_df["STATE"]=="CO"]
#USA_by_year_FL.plot.scatter(x="YEAR",y="RATE")#
x=USA_by_year_CO["YEAR"]
y=USA_by_year_CO["RATE"]

correlation = st.pearsonr(x,y)
(slope, intercept, rvalue, pvalue, stderr) = linregress(x, y)
regress_values = x * slope + intercept
line_equation = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x,y,c="#2ca02c")#green
plt.plot(x, y, "r-")

#California
USA_by_year_CA = USA_by_year_df.loc[USA_by_year_df["STATE"]=="CA"]
#USA_by_year_CA.plot.scatter(x="YEAR",y="RATE")
x=USA_by_year_CA["YEAR"]
y=USA_by_year_CA["RATE"]
correlation = st.pearsonr(x,y)
(slope, intercept, rvalue, pvalue, stderr) = linregress(x, y)
regress_values = x * slope + intercept
line_equation = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x,y,c="#9467bd")#Purple
plt.plot(x, y, "r-")
plt.show()




In [ ]:
plt.savefig("Output/Fig8.png")


In [ ]:
# Calculate correlation between YEAR and RATE
correlation = USA_by_year_df['YEAR'].corr(USA_by_year_df['RATE'])
#correlation=USA_by_year_df.corr()

# Print the correlation coefficient
print("Correlation coefficient between YEAR and RATE:", correlation)

In [ ]:

# Group the data by year and calculate mean rate
state_means = USA_by_year_df.groupby(["YEAR"])["RATE"].mean()
state_mean_df = pd.DataFrame(state_means)
state_mean_df=state_mean_df.reset_index()
# Create a scatterplot of the data
# Set axis labels and title
plt.xlabel('Year')
plt.ylabel('Mean Suicide Rate-USA')
plt.title('Mean Suicide Rate by Year in USA')

x=state_mean_df["YEAR"]
y=state_mean_df["RATE"]

#computing linear regression and plotting for year and the sucide rate in USA
correlation = st.pearsonr(x,y)
(slope, intercept, rvalue, pvalue, stderr) = linregress(x, y)
regress_values = x * slope + intercept
line_equation = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x,y,c="#9467bd")#Purple
plt.plot(x, regress_values, "r-")
plt.annotate(line_equation,(0,0),fontsize=15,color="red")

# Display the plot
plt.show()
plt.savefig("Output/Fig9.png")

print(f"The r-squared is :correlation:",correlation)


